Escuela Politécnica Nacional<br>


Facultad de Ingeniería en Sistemas<br>

Ingeniería en Ciencias de la Computación<br>
<br>
RECUPERACION DE INFORMACIÓN - ICCD753<br>

<br>
Tema: Modelo Booleano de RI<br>
Fecha de Entrega: 05-01-2022 <br>
<br>
Integrantes: <br>
Luis Ernesto Almeida Zambrano<br>
            Edison Daniel Cabrera Pabón

Ejemplo Uno

In [9]:
import pandas
# módulo para leer el contenido del archivo desde un archivo csv

from contextlib import redirect_stdout
# módulo para redirigir la salida a un archivo de texto
terms = []
# lista para almacenar los términos presentes en los documentos

keys = []
# lista para almacenar los nombres de los documentos

vec_Dic = {}
# diccionario para almacenar el nombre del documento y el vector booleano 
# como lista
dicti = {}
# diccionario para almacenar el nombre del documento y los términos presentes 
# en él como un vector

dummy_List = []
# lista para realizar algunas operaciones y borrarlas

Función para leer y separar el nombre de los documentos y los términos presente en él en una lista separada del marco de datos y también crear un diccionario que tiene el nombre del documento como clave y los términos presentes en
como la lista de cadenas que es el valor de la clave.

In [10]:
def filter(documents, rows, cols):
	for i in range(rows):
		for j in range(cols):
			# atravesar el marco de datos

			if(j == 0):
				# la primera columna tiene el nombre del documento en el archivo csv
				keys.append(documents.loc[i].iat[j])
			else:
				dummy_List.append(documents.loc[i].iat[j])
				# lista ficticia para actualizar los términos en el diccionario

				if documents.loc[i].iat[j] not in terms:
					# agregue los términos a la lista si no está presente; de lo contrario, continúe
					terms.append(documents.loc[i].iat[j])

		copy = dummy_List.copy()
		# copiando la lista ficticia a una lista diferente

		dicti.update({documents.loc[i].iat[0]: copy})
		# agregar el par clave-valor a un diccionario

		dummy_List.clear()
		# borrar la lista ficticia

En esta función obtenemos una representación booleana de los términos presentes en el documentos en forma de listas, luego creamos un diccionario que contiene el nombre de los documentos como clave y el valor como la lista de valores booleanos representando los términos presentes en el documento

In [11]:
def bool_Representation(dicti, rows, cols):
	terms.sort()
	# clasificamos los elementos en orden alfabético por conveniencia, el orden
	# del término no hace ninguna diferencia

	for i in (dicti):
		# para cada documento en el diccionario, verificamos cada cadena presente en
		# la lista

		for j in terms:
			# si la cadena está presente en la lista, agregamos 1; de lo contrario, agregamos 0

			if j in dicti[i]:
				dummy_List.append(1)
			else:
				dummy_List.append(0)
			# agregando 1 o 0 para obtener la representación booleana
		copy = dummy_List.copy()
		# copiando la lista ficticia a una lista diferente

		vec_Dic.update({i: copy})
		# agregar el par clave-valor a un diccionario

		dummy_List.clear()
		# borrar la lista ficticia

En esta función representamos la consulta en forma de vector booleano

In [12]:
def query_Vector(query):
	qvect = []
	# vector de consulta que se devuelve al final de la función

	for i in terms:
		# si la palabra presente en la lista de términos también está presente 
		# en la consulta luego agregue 1 de lo contrario agregue 0

		if i in query:
			qvect.append(1)
		else:
			qvect.append(0)

	return qvect
	# devuelve el vector de consulta que se obtiene en forma booleana

En esta función hacemos la predicción con respecto a qué documento está relacionado a la consulta dada realizando las operaciones booleanas

In [13]:
def prediction(q_Vect):
	dictionary = {}
	listi = []
	count = 0
	# inicialización del diccionario, lista y una variable que es contador
	# requerida para realizar el cálculo

	term_Len = len(terms)
	# número de términos presentes en la lista de términos

	for i in vec_Dic:
		# para cada documento del diccionario que contenga los términos presentes 
		# en él vector booleano
		for t in range(term_Len):
			if(q_Vect[t] == vec_Dic[i][t]):
				# si las palabras presentes en la consulta también están presentes en el
				# documento o si las palabras presentes en la consulta también están 
				# ausentes en el documento

				count += 1
				# aumentar el valor de la variable contador en uno
				# la condición en la que las palabras presentes en el documento y 
				# ausentes en consulta, presente en consulta y ausente en documento 
				# no se considera válido

		dictionary.update({i: count})
		# actualización del diccionario aquí el nombre del documento es la clave 
		# y el contador calculado anteriormente es el valor
	
		count = 0
		# reinicialización de la variable contador a 0

	for i in dictionary:
		listi.append(dictionary[i])
		# aquí agregamos el valor de contador a la lista

	listi = sorted(listi, reverse=True)
	# ordenamos la lista en orden descendente que es necesario para clasificar los
	# documentos según la relevancia

	ans = ' '
	# variable para almacenar el nombre del documento que es más relevante

	with open('Salida.txt', 'w') as f:
		with redirect_stdout(f):
			# para redirigir la salida a un archivo de texto

			print("ranking of the documents")

			for count, i in enumerate(listi):
				key = check(dictionary, i)
				# Llamada a la función para obtener la clave cuando se conoce el valor
				if count == 0:
					ans = key
					# para almacenar el nombre del documento que es más relevante

				print(key, "rank is", count+1)
				# imprime el nombre del documento junto con su rango

				dictionary.pop(key)
				# quitar la clave del diccionario después de imprimir

			print(ans, "is the most relevant document for the given query")
			# para imprimir el nombre del documento que es más relevante

Función para devolver la clave cuando se conoce el valor

In [14]:
def check(dictionary, val):
	for key, value in dictionary.items():
		if(val == value):
			# si el valor dado es el mismo que el valor presente en el diccionario
			# devuelve la clave
			return key

Función Principal del Programa

In [15]:
def main():
	documents = pandas.read_csv(r'DocumentoUno.csv')
	# para leer los datos del archivo csv como un marco de datos
	rows = len(documents)
	# para obtener el número de filas

	cols = len(documents.columns)
	# para obtener el número de columnas

	filter(documents, rows, cols)
	# llamada de función para leer y separar el nombre de los documentos y 
	# los términos presentes en él en una lista separada del marco de datos y 
	# también crear un diccionario que tiene el nombre del documento como clave y 
	# los términos presentes en él como la lista de cadenas cual es el valor 
	# de la clave

	bool_Representation(dicti, rows, cols)
	# en esta función obtenemos una representación booleana de los términos 
	# presentes en los documentos en forma de listas, luego creamos un diccionario 
	# que contiene el nombre de los documentos como clave y valor como la lista 
	# de valores booleanos que representan los términos presentes en el documento

	print("Enter query")
	query = input()
	# para obtener la entrada de la consulta del usuario, la siguiente entrada 
	# hockey is a national sport
	# se proporciona para obtener la salida como en el archivo Salida.txt 

	query = query.split(' ')
	# dividir la consulta como una lista de cadenas

	q_Vect = query_Vector(query)
	# llamada de función para representar la consulta en forma de vector booleano

	prediction(q_Vect)
	# llamada a la función para hacer la predicción con respecto a qué documento 
	# está relacionado con la consulta dada realizando las operaciones booleanas

#ejecución de la función principal
main()


Enter query
hockey is a national sport
